# Data Engineering Capstone Project

## Enviroment setup

In [1]:
# Import necessary libraries
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import monotonically_increasing_id as mono_id
import configparser

In [2]:
# Read config file
config = configparser.ConfigParser()
config.read_file(open('dl.cfg'))

INPUT_DATA = config['LOCAL']['INPUT_DATA']
INPUT_DATA_VACCINES = config['LOCAL']['INPUT_DATA_VACCINES']
OUTPUT_DATA = config['LOCAL']['OUTPUT_DATA']
DATA_COLUMNS = config['COMMON']['DATA_COLUMNS']

In [3]:
# Spark session
spark = SparkSession \
        .builder\
        .getOrCreate()

## Step 1: Scope the Project and Gather Data
In this step, we’ll:

* Identify and gather the data we'll be using for our project (at least two sources and more than 1 million rows).
* Explain what end use cases we'd like to prepare the data for (e.g., analytics table, app back-end, source-of-truth database, etc.)

We choose the following datasets:
* Brazilian Government' dataset [COVID-19 population imunization program](https://dados.gov.br/dataset/covid-19-vacinacao/resource/ef3bd0b8-b605-474b-9ae5-c97390c197a8?inner_span=True)

In [4]:
vaccines_df = spark.read.csv(INPUT_DATA_VACCINES, sep=';', header=True)

vaccines_df.printSchema()

root
 |-- document_id: string (nullable = true)
 |-- paciente_id: string (nullable = true)
 |-- paciente_idade: string (nullable = true)
 |-- paciente_datanascimento: string (nullable = true)
 |-- paciente_enumsexobiologico: string (nullable = true)
 |-- paciente_racacor_codigo: string (nullable = true)
 |-- paciente_racacor_valor: string (nullable = true)
 |-- paciente_endereco_coibgemunicipio: string (nullable = true)
 |-- paciente_endereco_copais: string (nullable = true)
 |-- paciente_endereco_nmmunicipio: string (nullable = true)
 |-- paciente_endereco_nmpais: string (nullable = true)
 |-- paciente_endereco_uf: string (nullable = true)
 |-- paciente_endereco_cep: string (nullable = true)
 |-- paciente_nacionalidade_enumnacionalidade: string (nullable = true)
 |-- estabelecimento_valor: string (nullable = true)
 |-- estabelecimento_razaosocial: string (nullable = true)
 |-- estalecimento_nofantasia: string (nullable = true)
 |-- estabelecimento_municipio_codigo: string (nullable = 

## Step 2: Explore and Assess the Data
In this step we need:
* Explore the data to identify data quality issues, like missing values, duplicate data, etc.
* Document steps necessary to clean the data

In [5]:
# Read the data dictionary from JSON and extract the valid columns
col_names = pd.read_json(DATA_COLUMNS, typ='series')
valid_columns = col_names.index.to_list()
valid_columns

['paciente_id',
 'paciente_idade',
 'paciente_datanascimento',
 'paciente_enumsexobiologico',
 'paciente_endereco_nmpais',
 'paciente_endereco_uf',
 'paciente_endereco_nmmunicipio',
 'estabelecimento_razaosocial',
 'estalecimento_nofantasia',
 'estabelecimento_uf',
 'estabelecimento_municipio_nome',
 'vacina_categoria_codigo',
 'vacina_categoria_nome',
 'vacina_grupoatendimento_codigo',
 'vacina_grupoatendimento_nome',
 'vacina_fabricante_nome',
 'vacina_codigo',
 'vacina_nome',
 'vacina_dataaplicacao']

In [6]:
# Get the difference between the dataframe colums and the valid columns
columns_todrop = list(set(vaccines_df.columns) - set(valid_columns))

columns_todrop

['vacina_descricao_dose',
 'paciente_endereco_coibgemunicipio',
 'sistema_origem',
 'data_importacao_rnds',
 'paciente_nacionalidade_enumnacionalidade',
 'estabelecimento_municipio_codigo',
 'paciente_racacor_valor',
 'document_id',
 'paciente_racacor_codigo',
 'paciente_endereco_copais',
 'vacina_lote',
 'vacina_fabricante_referencia',
 'estabelecimento_valor',
 'paciente_endereco_cep',
 'id_sistema_origem']

In [7]:
# Remove unused columns from dataframe
vaccines_df = vaccines_df.drop(*columns_todrop)
vaccines_df.printSchema()

root
 |-- paciente_id: string (nullable = true)
 |-- paciente_idade: string (nullable = true)
 |-- paciente_datanascimento: string (nullable = true)
 |-- paciente_enumsexobiologico: string (nullable = true)
 |-- paciente_endereco_nmmunicipio: string (nullable = true)
 |-- paciente_endereco_nmpais: string (nullable = true)
 |-- paciente_endereco_uf: string (nullable = true)
 |-- estabelecimento_razaosocial: string (nullable = true)
 |-- estalecimento_nofantasia: string (nullable = true)
 |-- estabelecimento_municipio_nome: string (nullable = true)
 |-- estabelecimento_uf: string (nullable = true)
 |-- vacina_grupoatendimento_codigo: string (nullable = true)
 |-- vacina_grupoatendimento_nome: string (nullable = true)
 |-- vacina_categoria_codigo: string (nullable = true)
 |-- vacina_categoria_nome: string (nullable = true)
 |-- vacina_fabricante_nome: string (nullable = true)
 |-- vacina_dataaplicacao: string (nullable = true)
 |-- vacina_codigo: string (nullable = true)
 |-- vacina_nome

In [8]:
# Replace the null values
vaccines_df = vaccines_df.fillna(\
    {\
        'vacina_categoria_codigo': 0, \
        'vacina_categoria_nome': 'N/A', \
        'vacina_grupoatendimento_nome': 'N/A', \
        'paciente_enumsexobiologico': 'N/A',\
        'estalecimento_nofantasia': 'N/A'
    })

## Step 3: Define the Data Model
* Map out the conceptual data model and explain why you chose that model
* List the steps necessary to pipeline the data into the chosen data model

## Step 4: Run ETL to Model the Data
* Create the data pipelines and the data model
* Include a data dictionary
* Run data quality checks to ensure the pipeline ran as expected
	* Integrity constraints on the relational database (e.g., unique key, data type, etc.)
	* Unit tests for the scripts to ensure they are doing the right thing
	* Source/count checks to ensure completeness

In [9]:
# Create vaccines table and write parquet files
vaccines_df.createOrReplaceTempView("vaccines_table_DF")
vaccines_table_DF = spark.sql("""
    SELECT  DISTINCT vacina_codigo AS id, 
                     vacina_nome AS name, 
                     vacina_fabricante_nome AS supplier
    FROM vaccines_table_DF
    ORDER BY supplier
""")

vaccines_table_DF.printSchema()
vaccines_table_DF.show()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- supplier: string (nullable = true)

+---+--------------------+--------------------+
| id|                name|            supplier|
+---+--------------------+--------------------+
| 87|Vacina covid-19 -...|BioNTech/Fosun Ph...|
| 86|Covid-19-Coronava...|   FUNDACAO BUTANTAN|
| 86|Covid-19-Coronava...|FUNDACAO OSWALDO ...|
| 85|Vacina Covid-19 -...|FUNDACAO OSWALDO ...|
| 88|Vacina covid-19 -...|       Janssen-Cilag|
| 87|Vacina covid-19 -...| MINISTERIO DA SAUDE|
| 89|Covid-19-AstraZeneca| MINISTERIO DA SAUDE|
| 88|Vacina covid-19 -...| MINISTERIO DA SAUDE|
| 86|Covid-19-Coronava...|SERUM INSTITUTE O...|
| 85|Vacina Covid-19 -...|SINOVAC LIFE SCIE...|
+---+--------------------+--------------------+



In [10]:
vaccines_table_path = OUTPUT_DATA + "vaccines_table.parquet"
vaccines_table_DF.write.mode("overwrite").parquet(vaccines_table_path)
print("Writing Vaccines Table DONE.")

# Read parquet file back to Spark:
vaccines_table_DF = spark.read.parquet(vaccines_table_path)

Writing Vaccines Table DONE.


In [11]:
# Create Health Institution table and write parquet files
vaccines_df.createOrReplaceTempView("health_institution_table_DF")
health_institution_table_DF = spark.sql("""
    SELECT DISTINCT estalecimento_nofantasia AS name,
                    estabelecimento_razaosocial AS organization,
                    estabelecimento_uf AS state,
                    estabelecimento_municipio_nome AS city
    FROM health_institution_table_DF
    ORDER BY name
""")

health_institution_table_DF.withColumn("id", mono_id())
health_institution_table_DF.printSchema()
health_institution_table_DF.show()

root
 |-- nome: string (nullable = false)
 |-- razaosocial: string (nullable = true)
 |-- uf: string (nullable = true)
 |-- municipio: string (nullable = true)

+--------------------+--------------------+---+--------------------+
|                nome|         razaosocial| uf|           municipio|
+--------------------+--------------------+---+--------------------+
|AMBULATORIO MUNIC...|PREFEITURA MUNICI...| MA|     POCAO DE PEDRAS|
|C SAUDE BACELAR V...|PREFEITURA MUNICI...| MA|           PIRAPEMAS|
|CEADIM DE SERRANO...|MUNICIPIO DE SERR...| MA| SERRANO DO MARANHAO|
|CENTRO DE ATENDIM...|PREFEITURA MUNICI...| MA|       VARGEM GRANDE|
|CENTRO DE ATENDIM...|PREFEITURA MUNICI...| MA|  ITINGA DO MARANHAO|
|CENTRO DE ATENDIM...|PREFEITURA MUNICI...| MA|              MONCAO|
|CENTRO DE ESPECIA...|PREFEITURA MUNICI...| MA|     BARAO DE GRAJAU|
|CENTRO DE ESPECIA...|PREFEITURA MUNICI...| MA|            PINHEIRO|
|CENTRO DE PARTO N...|PREFEITURA MUNICI...| MA|          BURITICUPU|
|CENTRO DE 

In [12]:
health_institution_path = OUTPUT_DATA + "health_institution.parquet"
health_institution_table_DF.write.mode("overwrite").parquet(health_institution_path)
print("Writing Health Institution Table DONE.")

# Read parquet file back to Spark:
health_institution_table_DF = spark.read.parquet(health_institution_path)

Writing Health Institution Table DONE.


In [13]:
# Create Category table and write parquet files
vaccines_df.createOrReplaceTempView("category_table_DF")
category_table_DF = spark.sql("""
    SELECT DISTINCT vacina_categoria_codigo AS id,
                    vacina_categoria_nome AS name
            FROM category_table_DF
            ORDER BY name
""")

category_table_DF.printSchema()
category_table_DF.show()

root
 |-- id: string (nullable = false)
 |-- name: string (nullable = false)

+---+--------------------+
| id|                name|
+---+--------------------+
|  1|        Comorbidades|
|  2|        Faixa Etária|
|  4|Forças Armadas (m...|
|  5|Forças de Seguran...|
| 14|Funcionário do Si...|
| 21|           Gestantes|
| -1|                 N/A|
|114|              Outros|
| 11|Pessoas com Defic...|
|  3|Pessoas de 60 ano...|
| 12|Pessoas em Situaç...|
| 15|População Privada...|
|  7|     Povos Indígenas|
|  6|Povos e Comunidad...|
| 25|           Puérperas|
| 16|Trabalhadores Ind...|
| 13|Trabalhadores Por...|
|  8|Trabalhadores da ...|
| 94|Trabalhadores de ...|
|  9|Trabalhadores de ...|
+---+--------------------+
only showing top 20 rows



In [14]:
category_path = OUTPUT_DATA + "category.parquet"
category_table_DF.write.mode("overwrite").parquet(category_path)
print("Writing Category Table DONE.")

# Read parquet file back to Spark:
category_table_DF = spark.read.parquet(category_path)

Writing Category Table DONE.


In [15]:
# Create Population Groups table and write parquet files
vaccines_df.createOrReplaceTempView("population_group_table_DF")
population_group_table_DF = spark.sql("""
    SELECT DISTINCT vacina_grupoatendimento_codigo AS id,
                    vacina_grupoatendimento_nome AS name
            FROM population_group_table_DF
        ORDER BY name
""")

population_group_table_DF.printSchema()
population_group_table_DF.show()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = false)

+------+--------------------+
|    id|                name|
+------+--------------------+
|000929|Acadêmicos/estuda...|
|000931|Agente Comunitári...|
|000930|Agente de Combate...|
|001006|          Aquaviário|
|000922|   Assistente Social|
|000927|Auxiliar de Enfer...|
|000901|Auxiliar de Veter...|
|000932|Auxiliar em Saúde...|
|001001|               Aéreo|
|000903|           Biomédico|
|000902|             Biólogo|
|000501|      Bombeiro Civil|
|000502|    Bombeiro Militar|
|001002|        Caminhoneiro|
|000114|    Cirrose hepática|
|001003|Coletivo Rodoviár...|
|000904|Cozinheiro e Auxi...|
|000905|  Cuidador de Idosos|
|000103|   Diabetes Mellitus|
|000105|Doença Renal Crônica|
+------+--------------------+
only showing top 20 rows



In [16]:
population_group_path = OUTPUT_DATA + "population_group.parquet"
population_group_table_DF.write.mode("overwrite").parquet(population_group_path)
print("Writing Population Group Table DONE.")

# Read parquet file back to Spark:
population_group_table_DF = spark.read.parquet(population_group_path)

Writing Population Group Table DONE.


In [25]:
# Create Patient table and write parquet files
vaccines_df.createOrReplaceTempView("patient_table_DF")
patient_table_DF = spark.sql("""
    SELECT DISTINCT paciente_id AS id,
					paciente_idade AS age,
					paciente_datanascimento AS birthdate,
					paciente_enumsexobiologico AS gender,
					paciente_endereco_nmpais AS country,
					paciente_endereco_uf AS state,
					paciente_endereco_nmmunicipio AS city
			FROM patient_table_DF
			WHERE paciente_id IS NOT NULL
			ORDER BY id
""")

patient_table_DF.printSchema()
patient_table_DF.show()

root
 |-- id: string (nullable = true)
 |-- age: string (nullable = true)
 |-- birthdate: string (nullable = true)
 |-- gender: string (nullable = false)
 |-- country: string (nullable = true)
 |-- state: string (nullable = true)
 |-- city: string (nullable = true)

+--------------------+---+----------+------+-------+-----+--------------------+
|                  id|age| birthdate|gender|country|state|                city|
+--------------------+---+----------+------+-------+-----+--------------------+
|0000073e1d449a8f1...| 57|1963-11-03|     M| BRASIL|   MA|            SAO LUIS|
|00000c2cb8d0d6da8...| 38|1982-08-19|     F| BRASIL|   MA|          IMPERATRIZ|
|00000ee6399e2172e...| 39|1982-02-14|     F| BRASIL|   MA|            SAO LUIS|
|0000146ec1364d3a4...| 25|1995-09-15|     F| BRASIL|   MA|            SAO LUIS|
|0000196ea219c99b3...| 33|1988-03-22|     M| BRASIL|   MA|      PACO DO LUMIAR|
|00001f5e735f9336e...| 31|1990-04-29|     M| BRASIL|   PI|            TERESINA|
|000021cece03

In [26]:
patient_path = OUTPUT_DATA + "patient.parquet"
patient_table_DF.write.mode("overwrite").parquet(patient_path)
print("Writing Patient Table DONE.")

# Read parquet file back to Spark:
patient_table_DF = spark.read.parquet(patient_path)

Writing Patient Table DONE.
